In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from PIL import ImageGrab
import requests
import os
import subprocess
import time
from datetime import datetime

In [2]:

class crawling:
    
    def __init__(self):
        #self.html = ""
        #self.soup = ""
        # 탐자를 우회하기 위함.(http1.1 통신 기준으로 http header에 user-Agent를 속인다.)
        self.header ={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5)\AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
                        "Accept":"text/html,application/xhtml+xml,application/xml;\
                        q=0.9,imgwebp,*/*;q=0.8"}

        
    def parsing(self,url):
        html = requests.get(url,headers=self.header)
        #파싱
        soup = BeautifulSoup(html.content,"html.parser")
        return soup 
    
        
    def Naver(self,start_date,end_date,keyword,sort):
        #해당 사이트 주소 리스트
        self.href = []
        self.company_name =[]
        self.select_user_company = []
        self.start_date = start_date
        self.end_date = end_date
        self.keyword = keyword
        
        #접근 오류 해결하기 위함.
        requests.packages.urllib3.disable_warnings()
        page = 1
        s_from = self.start_date.replace(".","")
        e_to = self.end_date.replace(".","")
        #sort 0 : 관련도,  1 : 최신순 , 2 : 오래된 
      
    
        max_page = (10-1)*10+1
        
        while page <=max_page:
            
            url =  "https://search.naver.com/search.naver?where=news&query="+keyword+ "&sort="+str(sort)+"&ds=" + self.start_date + "&de=" + self.end_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
            
            soup = self.parsing(url)
            
            #._sp_each_url
            # ._sp_each_title
            new_title = soup.select("._sp_each_title")
            #뉴스 기사 주소 수집
            for urls in new_title:
                self.href.append(urls['href'])
           
            # 뉴스기사 회사 이름    
            company_names = soup.select("._sp_each_source ")
            for urls in company_names:
                #회사이름 뒤에 언론사 선정 붙음.
                self.company_name.append(urls.text.replace("언론사 선정",""))
             
            #//*[@id="sp_nws1"]/dl/dd[1]/a
            
            #_sp_each_url
            #네이버 페이지 전환 규칙
            page +=10
        
        for href,company in zip(self.href,self.company_name):
            #print(href,company)
            if company == "아시아경제":
                self.select_user_company.append(href)

       
 
        
        self.select_company()           

    
    def select_company(self):
        self.__options = webdriver.ChromeOptions()
        self.__options.add_argument('headless')
        self.__options.add_argument('window-size=1920x1080')
        self.__options.add_argument('--disable-gpu')
        self.__options.add_argument("lang=ko_KR") # 한국어!
        self.__options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
                
        #,options= self.__options 
        #C:/Users/khk37/Downloads/chromedriver_win32/chromedriver
        #C:/Users/user/Downloads/chromedriver_win32/chromedriver
        self.__driver = webdriver.Chrome('C:/Users/khk37/Downloads/chromedriver_win32/chromedriver') 
        
       # print(len(self.select_user_company))
        print(self.select_user_company)
        number =1
        
        for news in self.select_user_company:
            #self.__driver.implicitly_wait(3)
            self.__driver.get(news)
            
            #스크롤 down
            self.__driver.execute_script("window.scrollTo(0,2500);")
            iframes = self.__driver.find_elements_by_css_selector('iframe')
            self.__driver.implicitly_wait(3)
            
            #time.sleep(3)
                
            
            screenshot_name= "댓글"
            capture = False
            
            
            try:
                for iframe in iframes:
                        #print(iframe.get_attribute('title'))
                        if iframe.get_attribute('title') == 'livere':
                            
                            self.__driver.get(iframe.get_attribute("src"))
                            
                            self.__driver.implicitly_wait(5)
                            
                            # 더보기 버튼 클릭.
                            
                            for index in range(0,10):
                                
                                try:    
                                        #//*[@id="list"]/div[3]/button
                                        #//*[@id="list"]/div[5]/button
                                    button = '//*[@id="list"]/div[{}]/button'.format(index)
                                    self.__driver.find_element_by_xpath(button).click()
                                  
                                except:
                                    pass
                                    
                            
                            self.__driver.implicitly_wait(2)
                            divs = self.__driver.find_elements_by_css_selector('div')
                            
                            for i in divs:
                                
                                if i.get_attribute('class') == 'reply-content-wrapper':
                                    #self.__driver.implicitly_wait(1)
                                    #time.sleep(5)
                                    if not capture:
                                        self.__driver.save_screenshot(screenshot_name+str(number)+'.png')
                                        capture = True
                                        
                                    number+=1
                                    print(i.get_attribute('data-content'))
                                    
                                    capture = False
            except:
                    continue
        
      
                    #break
                    
        #self.__driver.quit()
        
        
    @property   
    def get_select_user_company(self):
        return self.select_user_company
                                

            
            
        
        




In [26]:
naver =crawling()
#self,start_date,end_start,keyword,sort
naver.Naver('2019.7.00','2019.9.01','심석희','0')



['https://view.asiae.co.kr/article/2019083017304084385', 'https://view.asiae.co.kr/article/2019082310060528531', 'https://view.asiae.co.kr/news/view.htm?idxno=2019060321045386858', 'http://view.asiae.co.kr/news/view.htm?idxno=2019040810272026701']
정말!
그러면 큰일인데~~
제정신 차려라. 아무리 잘하고 싶어도 악마에게 영혼을 팔고싶나 너도 한심하다.
손석희 이제 노망났구나
손석희요?ㅋㅋㅋ심석희욬ㅋㅋㅋ
야! 전명규에게 당한 애들 생각은 않해 봤냐! 이기적인 인간들!ㅋㅋㅋㅋㅋ
전명규교수는 조재범 코치를 위해 탄원서 쓰리고 심석희선수에게 강요까지 했던 사람입니다.

저런 사람을 위해 심석희 선수와 빙상수 선수들이 호소문 쓴 게 자의일꺼라 생각하는건지..아시아경제는 언론이면 이치에 맞는 상황인지도 깊게 취재해서 기사를 쓰세요.
개인적으로 자식을 둔 부모입장에서 너무 안타깝고 화가난다. 학교가 할일은 학생들의 교육과 행정을 위해 일을해야하는데  ..학생들이 전교수의 교육을 받고싶어 선택한 한체대가 학생들은 생각도하지않고 저런 조치를 내리다니.. 시간강사에게 제일 중요한 대학 선수시기를 보내라고 할줄알았으면 심석희도 한체대에 가지않았을꺼다. 학생들의 의견을 존중해야하는게 학교가 할일이다. 여론에 휘둘리지말고 정신차려라 대한민국 교육부야
왜그랬어!
그것 잘못놀렸다가 인생 조졌네?
그것도 서로 좋아서 해야지 강제로하면 더 좋더냐?
이놈하고 이놈 윗대가리 그놈도 처벌 해야지!!
아주 선수들을 개패듯이 때렸더만!!
정말!
그러면 큰일인데~~
제정신 차려라. 아무리 잘하고 싶어도 악마에게 영혼을 팔고싶나 너도 한심하다.
손석희 이제 노망났구나
손석희요?ㅋㅋㅋ심석희욬ㅋㅋㅋ
야! 전명규에게 당한 애들 생각은 않해 봤냐! 이기적인 인간들!ㅋㅋㅋㅋㅋ
전명규교수는 조재범 코치를 위해 탄원서 쓰리

In [108]:
from selenium.common import exceptions
import pandas as pd
import re

In [150]:
######################### 네이버 댓급 수집.

class Setting(object):
    
    def __init__(self):
        pass
    
    
class Ecexl(object):
    
    def __init__(self,data,name =datetime.today()):
       
        self._to_csv(data, str(name))
    
    def _to_csv(self,data,name):
        self.dataframe = pd.DataFrame(data)
        #self.dataframe.columns = ['years','company','year']
        self.dataframe.to_csv(name+".csv",encoding='euc_kr', mode='a')
        
        

class Text(object):
    
    def __init__(self,file_name):
        self.file_name = file_name
        
        
    def _write_text(self,text):
        print(type(text))
        with open(self.file_name,'a',encoding='utf8') as file:
            file.write(str(text)+"\n")
        
        
        
        
        
        
class crawling:
    
    def __init__(self):
        #self.html = ""
        #self.soup = ""
        # 탐자를 우회하기 위함.(http1.1 통신 기준으로 http header에 user-Agent를 속인다.)
        self.header ={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5)\AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
                        "Accept":"text/html,application/xhtml+xml,application/xml;\
                        q=0.9,imgwebp,*/*;q=0.8"}

    def parsing_html(self, source):
        soup = BeautifulSoup(source, "lxml")
        return soup 
        
        
    def parsing(self, url):
        html = requests.get(url,headers=self.header)
        #파싱
        soup = BeautifulSoup(html.content,"lxml")
        return soup 
    
        
    def Naver(self,start_date,end_date,keyword,sort):
        #해당 사이트 주소 리스트
        self.href = []
        self.company_name =[]
        self.select_user_company = []
        self.start_date = start_date
        self.end_date = end_date
        self.keyword = keyword
        
        #접근 오류 해결하기 위함.
        requests.packages.urllib3.disable_warnings()
        page = 1
        s_from = self.start_date.replace(".","")
        e_to = self.end_date.replace(".","")
        #sort 0 : 관련도,  1 : 최신순 , 2 : 오래된 
      
    
        max_page = (10-1)*10+1
        
        while page <=max_page:
            
            url =  "https://search.naver.com/search.naver?where=news&query="+keyword+ "&sort="+str(sort)+"&ds=" + self.start_date + "&de=" + self.end_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
            
            soup = self.parsing(url)
            
            #._sp_each_url
            # ._sp_each_title
            new_title = soup.select("._sp_each_url")
            
            company_names = soup.select("._sp_each_source")
            
            years = soup.select(".txt_inline")
        
   
            
            #뉴스 기사 주소 수집
            for urls, name, year in zip(new_title, company_names, years):
                self.href.append(urls['href'])
                try:
                    pattern = '\d+.(\d+).(\d+).' #정규표현식
                    r = re.compile(pattern)
                    match = r.search(year.text).group(0)


                    #print(name.text.replace("언론사 선정",""),urls['href'],match)
                    Ecexl({name.text.replace("언론사 선정",""),urls['href'],match},'justTT')
                    
                except Exception as e:
                    print(e)
                   # continue

           
            #네이버 페이지 전환 규칙
            page +=10
        
        self.select_company()           


      
    
    def select_company(self):
        self.__options = webdriver.ChromeOptions()
        self.__options.add_argument('headless')
        self.__options.add_argument('window-size=1920x1080')
        self.__options.add_argument('--disable-gpu')
        self.__options.add_argument("lang=ko_KR") # 한국어!
        self.__options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
                
        #,options= self.__options 
        #C:/Users/khk37/Downloads/chromedriver_win32/chromedriver
        #C:/Users/user/Downloads/chromedriver_win32/chromedriver
        self.__driver = webdriver.Chrome('C:/Users/khk37/Downloads/chromedriver_win32/chromedriver') 
        
       # print(len(self.select_user_company))
        number =1
        collect_text = Text('collect_text.txt')
        for news in self.href:
            self.__driver.implicitly_wait(3)
            self.__driver.get(news)
            
            comment_url = news[:37]+"m_view=1&includeAllCount=true&"+news[39:]
            
            self.__driver.find_element_by_css_selector(".u_cbox_btn_view_comment").click()
            
            self.__driver.implicitly_wait(3)
            #self.__driver.get(comment_url)
            page = 0
           
            try:
                while True:
                      self.__driver.find_element_by_css_selector(".u_cbox_btn_more").click()
                      #self.__driver.find_element_by_xpath('//*[@id="cbox_module"]/div[2]/div[9]/a').click()
                      time.sleep(3)
                      page +=1
                      print(page)
                     #//*[@id="cbox_module"]/div[2]/div[9]/a
                    
            
            except exceptions.ElementNotVisibleException as e: # 페이지 끝
                            pass
    
            except Exception as e: # 다른 예외 발생시 확인
                            print("에러 :  ",e)
                            
            
            soup = self.parsing_html(self.__driver.page_source)
            comment_list = soup.find_all("span", {"class" : "u_cbox_contents"})
            
            
            for comment in comment_list:
                  collect_text._write_text(comment) 
            #  print(comment.text)
              #  print(" ")
                
                
            


In [151]:

test = crawling()
test.Naver('2019.7.00','2019.9.01','심석희','0')


'NoneType' object has no attribute 'group'
1
2
3
4
5
6
7
에러 :   Message: element not interactable
  (Session info: chrome=76.0.3809.132)

<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<cla

1
2
3
4
5
6
7
8
9
에러 :   Message: element not interactable
  (Session info: chrome=76.0.3809.132)

<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.eleme

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".u_cbox_btn_view_comment"}
  (Session info: chrome=76.0.3809.132)
